In [ ]:
! pip install llama_index llama-index-embeddings-huggingface llama-index-vector-stores-qdrant 

In [ ]:
! pip install llama-index-llms-gemini 

In [ ]:
! pip install -q llama-index google-generativeai 

In [ ]:
import os
import sys
import pprint
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
    ServiceContext,
    Document
)
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.text_splitter import SenteceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.schema import MetadataMode
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
import google.generativeai as genai
from llama_index.llms.gemini import Gemini
from llama_index.core.llms import ChatMessage


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Loading embedding model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2", max_length=512)

In [ ]:
GOOGLE_API_KEY = "GOOGLE_API_KEY" # Add your api key here
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

In [ ]:
llm = Gemini(model="models/gimini-pro")

In [ ]:
messages = [
    ChatMessage(role="user", content="Hello friend!"),
    # ChatMessage(role="assistant", content="Yarr what is shakin' matey?"),
    # ChatMessage(role="user", content="Help me decide what to have for dinner.")
]

response = llm.chat(messages)
print(response)

In [ ]:
"""# create local directory and retrieve file from external source
!mkdir -p 'my_data'
!wget 'https://www.gutenberg.org/cache/epub/72306/pg72306.txt' -0 './my_data/teahistory.txt'
!wget 'https://www.gutenberg.org/cache/epub/11367/pg11367.txt' -0 './my_data/chinahistory.txt'

"""

In [ ]:
documents = SimpleDirectoryReader(input_files=["/content/my_data/got_book.txt"]).load_data()

In [ ]:
documents

In [ ]:
# Inspect documents
print("length of doc: " + str(len(documents)))
print("-----")
# pprint(documents)

In [ ]:
documents[0].metadata